# Constant Demand 


In [292]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [293]:
df1 = pd.read_csv('../Output-Files/constant-Demand.csv')
df2 = pd.read_csv('../Output-Files/linear-Demand.csv')

In [294]:
first_batch_ready = 10 # October 2024

Red_Wine_1_production_time = 19
White_Wine_1_production_time = 4
Red_Wine_2_production_time = 28
White_Wine_2_production_time = 16

start_date = datetime(2024, 1, 1)

## Simulation 1

In [295]:
# Initialize a new DataFrame to hold the adjusted demands
df1_sim1 = df1.copy()

# Convert "Month" column to datetime for easier comparison
df1_sim1["Month"] = pd.to_datetime(df1_sim1["Month"], format="%b-%Y")

# Define the readiness date for White and Red wine
white_wine_ready_date = datetime(2025, 2, 1)  # February 2025
red_wine_ready_date = datetime(2026, 5, 1)    # May 2026

# Initialize columns for adjusted demands
for wine_type in ['Red', 'White']:
    for country in ["Singapore", "Thailand", "Japan", "Korea", "Indonesia"]:
        df1_sim1[f"{wine_type}_{country}"] = 0.0

# Split the demand for Red and White wine
for i, row in df1_sim1.iterrows():
    current_month = row["Month"]
    for country in ["Singapore", "Thailand", "Japan", "Korea", "Indonesia"]:
        # Halve the original demand since 50% demand for each wine type
        half_demand = row[country] / 2
        
        # Adjust demand for White wine if current month is after white wine readiness date
        if current_month >= white_wine_ready_date:
            df1_sim1.at[i, f"White_{country}"] = half_demand
            
        # Adjust demand for Red wine if current month is after red wine readiness date
        if current_month >= red_wine_ready_date:
            df1_sim1.at[i, f"Red_{country}"] = half_demand

# Drop original country columns as they're now split into Red and White demands
df1_sim1.drop(["Singapore", "Thailand", "Japan", "Korea", "Indonesia"], axis=1, inplace=True)

# Display a sample of the adjusted DataFrame to verify correctness
df1_sim1


,Month,Red_Singapore,Red_Thailand,Red_Japan,Red_Korea,Red_Indonesia,White_Singapore,White_Thailand,White_Japan,White_Korea,White_Indonesia
0,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2024-02-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2024-03-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2024-04-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2024-05-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
115,2033-08-01,0.006500,0.004184,0.002498,0.003307,0.002397,0.006500,0.004184,0.002498,0.003307,0.002397
116,2033-09-01,0.006378,0.004106,0.002452,0.003245,0.002352,0.006378,0.004106,0.002452,0.003245,0.002352
117,2033-10-01,0.006333,0.004077,0.002434,0.003222,0.002335,0.006333,0.004077,0.002434,0.003222,0.002335
118,2033-11-01,0.006378,0.004106,0.002452,0.003245,0.002352,0.006378,0.004106,0.002452,0.003245,0.002352


In [296]:
df = df1_sim1.copy()

# Aggregating the total demand for Red and White wine across all months for each country
red_columns = [col for col in df.columns if 'Red' in col]
white_columns = [col for col in df.columns if 'White' in col]

# Summing up the demand for Red and White wine for each country
total_red_demand = df[red_columns].sum().sum()
total_white_demand = df[white_columns].sum().sum()

# Convert 'Month' to datetime for proper sorting and display
df['Month'] = pd.to_datetime(df['Month'])

# Aggregate monthly total demand for Red and White wine across all countries
df['Total_Red_Demand'] = df[red_columns].sum(axis=1)
df['Total_White_Demand'] = df[white_columns].sum(axis=1)
df['Total_Wine_Demand'] = df['Total_Red_Demand'] + df['Total_White_Demand']

# Extracting year-month for the x-axis
df['Year_Month'] = df['Month'].dt.strftime('%Y-%m')

# Creating separate traces for Red and White wine demand over time
fig = go.Figure()

# Red wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_Red_Demand']*1000,
    name="Red Wine Demand",
    marker_color="red"
))

# White wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_White_Demand']*1000,
    name="White Wine Demand",
    marker_color="lightgreen"
))

fig.update_layout(
    title="Constant Demand: Monthly Wine Demand Satisfied by Type",
    xaxis_title="Month-Year",
    yaxis_title="Wine Demand Satisfied (in Thousands)",
    xaxis={'type': 'category'},  # Ensures that months are treated as distinct categories
    barmode='stack'  # Groups the bars for red and white wine side by side for each month
)

# This setup would allow for filtering by wine type by clicking on the legend items.
fig.show()


In [297]:
# Assuming df is your DataFrame containing the data
df = df1_sim1.copy()

# Convert 'Month' to datetime for proper sorting and display
df['Month'] = pd.to_datetime(df['Month'])

# Aggregate monthly total demand for Red and White wine across all countries
df['Total_Red_Demand'] = df[red_columns].sum(axis=1)
df['Total_White_Demand'] = df[white_columns].sum(axis=1)

# Calculate cumulative sum for each type of wine
df['Cumulative_Red_Demand'] = df['Total_Red_Demand'].cumsum() * 125 # 'Red_Wine_1': 125,
df['Cumulative_White_Demand'] = df['Total_White_Demand'].cumsum() * 90 # 'White_Wine_1': 90,

# Create a list of months
months = list(range(1, len(df) + 1))

# Create stacked line plot
fig = go.Figure()

# Stacked red wine demand
fig.add_trace(go.Scatter(
    x=months,
    y=df['Cumulative_Red_Demand'],
    mode='lines',
    name='Cumulative Red Wine Demand',
    stackgroup='one',
    marker_color='darkred'
))

# Stacked white wine demand
fig.add_trace(go.Scatter(
    x=months,
    y=df['Cumulative_White_Demand'],
    mode='lines',
    name='Cumulative White Wine Demand',
    stackgroup='one',
    marker_color='lightgreen'
))

fig.update_layout(
    title="Constant Demand: Expected Profit in Simulation 1",
    xaxis_title="Month",
    yaxis_title="Expected Profit in Euros (in Millions)",
    hovermode="x unified",
    xaxis={'type': 'category'},  #
)

fig.show()


## Simulation 2

In [298]:
# Initialize a new DataFrame to hold the adjusted demands
df1_sim2 = df1.copy()

# Convert "Month" column to datetime for easier comparison
df1_sim2["Month"] = pd.to_datetime(df1_sim2["Month"], format="%b-%Y")

# Calculate the readiness dates based on production times
white_wine_1_ready_date = start_date + timedelta(days=(first_batch_ready + White_Wine_1_production_time) * 30)
red_wine_1_ready_date = start_date + timedelta(days=(first_batch_ready + Red_Wine_1_production_time) * 30)
white_wine_2_ready_date = start_date + timedelta(days=(first_batch_ready + White_Wine_2_production_time) * 30)
red_wine_2_ready_date = start_date + timedelta(days=(first_batch_ready + Red_Wine_2_production_time) * 30)

# Initialize columns for adjusted demands
for wine_type in ['Red_1', 'White_1', 'Red_2', 'White_2']:
    for country in ["Singapore", "Thailand", "Japan", "Korea", "Indonesia"]:
        df1_sim2[f"{wine_type}_{country}"] = 0.0

# Split the demand for each wine type
for i, row in df1_sim2.iterrows():
    current_month = row["Month"]
    for country in ["Singapore", "Thailand", "Japan", "Korea", "Indonesia"]:
        original_demand = row[country]
        
        # Calculate demands for each wine type based on their contributions
        demand_red_1 = original_demand * 0.35
        demand_white_1 = original_demand * 0.35
        demand_red_2 = original_demand * 0.15
        demand_white_2 = original_demand * 0.15
        
        # Adjust demand based on the readiness date of each wine type
        if current_month >= white_wine_1_ready_date:
            df1_sim2.at[i, f"White_1_{country}"] = demand_white_1
            
        if current_month >= red_wine_1_ready_date:
            df1_sim2.at[i, f"Red_1_{country}"] = demand_red_1
            
        if current_month >= white_wine_2_ready_date:
            df1_sim2.at[i, f"White_2_{country}"] = demand_white_2
            
        if current_month >= red_wine_2_ready_date:
            df1_sim2.at[i, f"Red_2_{country}"] = demand_red_2

# Drop original country columns as they're now split into specific wine demands
df1_sim2.drop(["Singapore", "Thailand", "Japan", "Korea", "Indonesia"], axis=1, inplace=True)

# Display a sample of the adjusted DataFrame to verify correctness
df1_sim2


,Month,Red_1_Singapore,Red_1_Thailand,Red_1_Japan,Red_1_Korea,Red_1_Indonesia,White_1_Singapore,White_1_Thailand,White_1_Japan,White_1_Korea,...,Red_2_Singapore,Red_2_Thailand,Red_2_Japan,Red_2_Korea,Red_2_Indonesia,White_2_Singapore,White_2_Thailand,White_2_Japan,White_2_Korea,White_2_Indonesia
0,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2024-02-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2024-03-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2024-04-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2024-05-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2033-08-01,0.004550,0.002929,0.001749,0.002315,0.001678,0.004550,0.002929,0.001749,0.002315,...,0.001950,0.001255,0.000750,0.000992,0.000719,0.001950,0.001255,0.000750,0.000992,0.000719
116,2033-09-01,0.004465,0.002874,0.001716,0.002271,0.001646,0.004465,0.002874,0.001716,0.002271,...,0.001913,0.001232,0.000735,0.000973,0.000706,0.001913,0.001232,0.000735,0.000973,0.000706
117,2033-10-01,0.004433,0.002854,0.001704,0.002255,0.001635,0.004433,0.002854,0.001704,0.002255,...,0.001900,0.001223,0.000730,0.000967,0.000701,0.001900,0.001223,0.000730,0.000967,0.000701
118,2033-11-01,0.004465,0.002874,0.001716,0.002271,0.001646,0.004465,0.002874,0.001716,0.002271,...,0.001913,0.001232,0.000735,0.000973,0.000706,0.001913,0.001232,0.000735,0.000973,0.000706


In [299]:
df = df1_sim2.copy()

# Aggregating the total demand for Red and White wine across all months for each country
red_columns_1 = [col for col in df.columns if 'Red_1' in col]
white_columns_1 = [col for col in df.columns if 'White_1' in col]
red_columns_2 = [col for col in df.columns if 'Red_2' in col]
white_columns_2 = [col for col in df.columns if 'White_2' in col]

# Summing up the demand for Red and White wine for each country
total_red_demand_1 = df[red_columns_1].sum().sum()
total_white_demand_1 = df[white_columns_1].sum().sum()
total_red_demand_2 = df[red_columns_2].sum().sum()
total_white_demand_2 = df[white_columns_2].sum().sum()

# Convert 'Month' to datetime for proper sorting and display
df['Month'] = pd.to_datetime(df['Month'])

# Aggregate monthly total demand for Red and White wine across all countries
df['Total_Red_Demand_1'] = df[red_columns_1].sum(axis=1)
df['Total_White_Demand_1'] = df[white_columns_1].sum(axis=1)
df['Total_Red_Demand_2'] = df[red_columns_2].sum(axis=1)
df['Total_White_Demand_2'] = df[white_columns_2].sum(axis=1)

df['Total_Wine_Demand'] = df['Total_Red_Demand_1'] + df['Total_White_Demand_1'] + df['Total_Red_Demand_2'] + df['Total_White_Demand_2']

# Extracting year-month for the x-axis
df['Year_Month'] = df['Month'].dt.strftime('%Y-%m')

# Creating separate traces for Red and White wine demand over time
fig = go.Figure()

# Red wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_Red_Demand_1']*1000,
    name="Red Wine Demand",
    marker_color="red"
))

# White wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_White_Demand_1']*1000,
    name="White Wine Demand",
    marker_color="lightgreen"
))

# Red wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_Red_Demand_2']*1000,
    name="Red Wine Demand",
    marker_color="darkred"
))

# White wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_White_Demand_2']*1000,
    name="White Wine Demand",
    marker_color="green"
))

fig.update_layout(
    title="Constant Demand: Monthly Wine Demand Satisfied by Type",
    xaxis_title="Month-Year",
    yaxis_title="Wine Demand Satisfied (in Thousands)",
    xaxis={'type': 'category'},  # Ensures that months are treated as distinct categories
    barmode='stack'  # Groups the bars for red and white wine side by side for each month
)

# This setup would allow for filtering by wine type by clicking on the legend items.
fig.show()


In [300]:

# Assuming df is your DataFrame containing the data
df = df1_sim2.copy()

# Aggregating the total demand for Red and White wine across all months for each country
red_columns_1 = [col for col in df.columns if 'Red_1' in col]
white_columns_1 = [col for col in df.columns if 'White_1' in col]
red_columns_2 = [col for col in df.columns if 'Red_2' in col]
white_columns_2 = [col for col in df.columns if 'White_2' in col]

# Convert 'Month' to datetime for proper sorting and display
df['Month'] = pd.to_datetime(df['Month'])

# Aggregate monthly total demand for Red and White wine across all countries
df['Total_Red_Demand_1'] = df[red_columns_1].sum(axis=1)
df['Total_White_Demand_1'] = df[white_columns_1].sum(axis=1)
df['Total_Red_Demand_2'] = df[red_columns_2].sum(axis=1)
df['Total_White_Demand_2'] = df[white_columns_2].sum(axis=1)

# Calculate cumulative sum for each type of wine
df['Cumulative_Red_Demand_1'] = df['Total_Red_Demand_1'].cumsum() * 125 # 'Red_Wine_1': 125,
df['Cumulative_White_Demand_1'] = df['Total_White_Demand_1'].cumsum() * 90 # 'White_Wine_1': 90,
df['Cumulative_Red_Demand_2'] = df['Total_Red_Demand_2'].cumsum() * 260 # 'Red_Wine_2': 260,
df['Cumulative_White_Demand_2'] = df['Total_White_Demand_2'].cumsum() * 200 # 'White_Wine_2': 200,

# Extracting year-month for the x-axis
df['Year_Month'] = df['Month'].dt.strftime('%Y-%m')

# Create stacked line plot
fig = go.Figure()

# Stacked red wine demand 1
fig.add_trace(go.Scatter(
    x=df['Year_Month'],
    y=df['Cumulative_Red_Demand_1'],
    mode='lines',
    name='Cumulative Red Wine Demand 1',
    stackgroup='one',
    line=dict(color='red')
))

# Stacked white wine demand 1
fig.add_trace(go.Scatter(
    x=df['Year_Month'],
    y=df['Cumulative_White_Demand_1'],
    mode='lines',
    name='Cumulative White Wine Demand 1',
    stackgroup='one',
    line=dict(color='lightgreen')
))

# Stacked red wine demand 2
fig.add_trace(go.Scatter(
    x=df['Year_Month'],
    y=df['Cumulative_Red_Demand_2'],
    mode='lines',
    name='Cumulative Red Wine Demand 2',
    stackgroup='one',
    line=dict(color='darkred')
))

# Stacked white wine demand 2
fig.add_trace(go.Scatter(
    x=df['Year_Month'],
    y=df['Cumulative_White_Demand_2'],
    mode='lines',
    name='Cumulative White Wine Demand 2',
    stackgroup='one',
    line=dict(color='green')
))

fig.update_layout(
    title="Constant Demand: Expected Profit in Simulation 2",
    xaxis_title="Month",
    yaxis_title="Expected Profit in Euros (in Millions)",
    hovermode="x unified",
    xaxis={'type': 'category'},  #
)

fig.show()

# Linear Demand 

## Simulation 1

In [301]:
# Initialize a new DataFrame to hold the adjusted demands
df2_sim1 = df2.copy()

# Convert "Month" column to datetime for easier comparison
df2_sim1["Month"] = pd.to_datetime(df2_sim1["Month"], format="%b-%Y")

# Define the readiness date for White and Red wine
white_wine_ready_date = datetime(2025, 2, 1)  # February 2025
red_wine_ready_date = datetime(2026, 5, 1)    # May 2026

# Initialize columns for adjusted demands
for wine_type in ['Red', 'White']:
    for country in ["Singapore", "Thailand", "Japan", "Korea", "Indonesia"]:
        df2_sim1[f"{wine_type}_{country}"] = 0.0

# Split the demand for Red and White wine
for i, row in df2_sim1.iterrows():
    current_month = row["Month"]
    for country in ["Singapore", "Thailand", "Japan", "Korea", "Indonesia"]:
        # Halve the original demand since 50% demand for each wine type
        half_demand = row[country] / 2
        
        # Adjust demand for White wine if current month is after white wine readiness date
        if current_month >= white_wine_ready_date:
            df2_sim1.at[i, f"White_{country}"] = half_demand
            
        # Adjust demand for Red wine if current month is after red wine readiness date
        if current_month >= red_wine_ready_date:
            df2_sim1.at[i, f"Red_{country}"] = half_demand

# Drop original country columns as they're now split into Red and White demands
df2_sim1.drop(["Singapore", "Thailand", "Japan", "Korea", "Indonesia"], axis=1, inplace=True)

# Display a sample of the adjusted DataFrame to verify correctness
df2_sim1

,Month,Red_Singapore,Red_Thailand,Red_Japan,Red_Korea,Red_Indonesia,White_Singapore,White_Thailand,White_Japan,White_Korea,White_Indonesia
0,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2024-02-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2024-03-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2024-04-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2024-05-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
115,2033-08-01,0.008929,0.006159,0.003187,0.003894,0.003293,0.008929,0.006159,0.003187,0.003894,0.003293
116,2033-09-01,0.008867,0.006116,0.003165,0.003867,0.003270,0.008867,0.006116,0.003165,0.003867,0.003270
117,2033-10-01,0.008929,0.006159,0.003187,0.003894,0.003293,0.008929,0.006159,0.003187,0.003894,0.003293
118,2033-11-01,0.009100,0.006277,0.003248,0.003968,0.003356,0.009100,0.006277,0.003248,0.003968,0.003356


In [302]:
df = df2_sim1.copy()

# Aggregating the total demand for Red and White wine across all months for each country
red_columns = [col for col in df.columns if 'Red' in col]
white_columns = [col for col in df.columns if 'White' in col]

# Summing up the demand for Red and White wine for each country
total_red_demand = df[red_columns].sum().sum()
total_white_demand = df[white_columns].sum().sum()

# Convert 'Month' to datetime for proper sorting and display
df['Month'] = pd.to_datetime(df['Month'])

# Aggregate monthly total demand for Red and White wine across all countries
df['Total_Red_Demand'] = df[red_columns].sum(axis=1)
df['Total_White_Demand'] = df[white_columns].sum(axis=1)
df['Total_Wine_Demand'] = df['Total_Red_Demand'] + df['Total_White_Demand']

# Extracting year-month for the x-axis
df['Year_Month'] = df['Month'].dt.strftime('%Y-%m')

# Creating separate traces for Red and White wine demand over time
fig = go.Figure()

# Red wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_Red_Demand']*1000,
    name="Red Wine Demand",
    marker_color="red"
))

# White wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_White_Demand']*1000,
    name="White Wine Demand",
    marker_color="lightgreen"
))

fig.update_layout(
    title="Linear Demand: Monthly Wine Demand Satisfied by Type",
    xaxis_title="Month-Year",
    yaxis_title="Wine Demand Satisfied (in Thousands)",
    xaxis={'type': 'category'},  # Ensures that months are treated as distinct categories
    barmode='stack'  # Groups the bars for red and white wine side by side for each month
)

# This setup would allow for filtering by wine type by clicking on the legend items.
fig.show()


In [303]:
# Assuming df is your DataFrame containing the data
df = df2_sim1.copy()

# Convert 'Month' to datetime for proper sorting and display
df['Month'] = pd.to_datetime(df['Month'])

# Aggregate monthly total demand for Red and White wine across all countries
df['Total_Red_Demand'] = df[red_columns].sum(axis=1)
df['Total_White_Demand'] = df[white_columns].sum(axis=1)

# Calculate cumulative sum for each type of wine
df['Cumulative_Red_Demand'] = df['Total_Red_Demand'].cumsum() * 125 # 'Red_Wine_1': 125,
df['Cumulative_White_Demand'] = df['Total_White_Demand'].cumsum() * 90 # 'White_Wine_1': 90,

# Create a list of months
months = list(range(1, len(df) + 1))

# Create stacked line plot
fig = go.Figure()

# Stacked red wine demand
fig.add_trace(go.Scatter(
    x=months,
    y=df['Cumulative_Red_Demand'],
    mode='lines',
    name='Cumulative Red Wine Demand',
    stackgroup='one',
    marker_color='darkred'
))

# Stacked white wine demand
fig.add_trace(go.Scatter(
    x=months,
    y=df['Cumulative_White_Demand'],
    mode='lines',
    name='Cumulative White Wine Demand',
    stackgroup='one',
    marker_color='lightgreen'
))

fig.update_layout(
    title="Linear Demand: Expected Profit in Simulation 1",
    xaxis_title="Month",
    yaxis_title="Expected Profit in Euros (in Millions)",
    hovermode="x unified"
)

fig.show()

## Simulation 2

In [304]:
# Initialize a new DataFrame to hold the adjusted demands
df2_sim2 = df2.copy()

# Convert "Month" column to datetime for easier comparison
df2_sim2["Month"] = pd.to_datetime(df2_sim2["Month"], format="%b-%Y")

# Calculate the readiness dates based on production times
white_wine_1_ready_date = start_date + timedelta(days=(first_batch_ready + White_Wine_1_production_time) * 30)
red_wine_1_ready_date = start_date + timedelta(days=(first_batch_ready + Red_Wine_1_production_time) * 30)
white_wine_2_ready_date = start_date + timedelta(days=(first_batch_ready + White_Wine_2_production_time) * 30)
red_wine_2_ready_date = start_date + timedelta(days=(first_batch_ready + Red_Wine_2_production_time) * 30)

# Initialize columns for adjusted demands
for wine_type in ['Red_1', 'White_1', 'Red_2', 'White_2']:
    for country in ["Singapore", "Thailand", "Japan", "Korea", "Indonesia"]:
        df2_sim2[f"{wine_type}_{country}"] = 0.0

# Split the demand for each wine type
for i, row in df2_sim2.iterrows():
    current_month = row["Month"]
    for country in ["Singapore", "Thailand", "Japan", "Korea", "Indonesia"]:
        original_demand = row[country]
        
        # Calculate demands for each wine type based on their contributions
        demand_red_1 = original_demand * 0.35
        demand_white_1 = original_demand * 0.35
        demand_red_2 = original_demand * 0.15
        demand_white_2 = original_demand * 0.15
        
        # Adjust demand based on the readiness date of each wine type
        if current_month >= white_wine_1_ready_date:
            df2_sim2.at[i, f"White_1_{country}"] = demand_white_1
            
        if current_month >= red_wine_1_ready_date:
            df2_sim2.at[i, f"Red_1_{country}"] = demand_red_1
            
        if current_month >= white_wine_2_ready_date:
            df2_sim2.at[i, f"White_2_{country}"] = demand_white_2
            
        if current_month >= red_wine_2_ready_date:
            df2_sim2.at[i, f"Red_2_{country}"] = demand_red_2

# Drop original country columns as they're now split into specific wine demands
df2_sim2.drop(["Singapore", "Thailand", "Japan", "Korea", "Indonesia"], axis=1, inplace=True)

# Display a sample of the adjusted DataFrame to verify correctness
df2_sim2

,Month,Red_1_Singapore,Red_1_Thailand,Red_1_Japan,Red_1_Korea,Red_1_Indonesia,White_1_Singapore,White_1_Thailand,White_1_Japan,White_1_Korea,...,Red_2_Singapore,Red_2_Thailand,Red_2_Japan,Red_2_Korea,Red_2_Indonesia,White_2_Singapore,White_2_Thailand,White_2_Japan,White_2_Korea,White_2_Indonesia
0,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2024-02-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2024-03-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2024-04-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2024-05-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2033-08-01,0.006250,0.004311,0.002231,0.002726,0.002305,0.006250,0.004311,0.002231,0.002726,...,0.002679,0.001848,0.000956,0.001168,0.000988,0.002679,0.001848,0.000956,0.001168,0.000988
116,2033-09-01,0.006207,0.004281,0.002215,0.002707,0.002289,0.006207,0.004281,0.002215,0.002707,...,0.002660,0.001835,0.000949,0.001160,0.000981,0.002660,0.001835,0.000949,0.001160,0.000981
117,2033-10-01,0.006250,0.004311,0.002231,0.002726,0.002305,0.006250,0.004311,0.002231,0.002726,...,0.002679,0.001848,0.000956,0.001168,0.000988,0.002679,0.001848,0.000956,0.001168,0.000988
118,2033-11-01,0.006370,0.004394,0.002274,0.002778,0.002349,0.006370,0.004394,0.002274,0.002778,...,0.002730,0.001883,0.000974,0.001190,0.001007,0.002730,0.001883,0.000974,0.001190,0.001007


In [305]:
df = df2_sim2.copy()

# Aggregating the total demand for Red and White wine across all months for each country
red_columns_1 = [col for col in df.columns if 'Red_1' in col]
white_columns_1 = [col for col in df.columns if 'White_1' in col]
red_columns_2 = [col for col in df.columns if 'Red_2' in col]
white_columns_2 = [col for col in df.columns if 'White_2' in col]

# Summing up the demand for Red and White wine for each country
total_red_demand_1 = df[red_columns_1].sum().sum()
total_white_demand_1 = df[white_columns_1].sum().sum()
total_red_demand_2 = df[red_columns_2].sum().sum()
total_white_demand_2 = df[white_columns_2].sum().sum()

# Convert 'Month' to datetime for proper sorting and display
df['Month'] = pd.to_datetime(df['Month'])

# Aggregate monthly total demand for Red and White wine across all countries
df['Total_Red_Demand_1'] = df[red_columns_1].sum(axis=1)
df['Total_White_Demand_1'] = df[white_columns_1].sum(axis=1)
df['Total_Red_Demand_2'] = df[red_columns_2].sum(axis=1)
df['Total_White_Demand_2'] = df[white_columns_2].sum(axis=1)

df['Total_Wine_Demand'] = df['Total_Red_Demand_1'] + df['Total_White_Demand_1'] + df['Total_Red_Demand_2'] + df['Total_White_Demand_2']

# Extracting year-month for the x-axis
df['Year_Month'] = df['Month'].dt.strftime('%Y-%m')

# Creating separate traces for Red and White wine demand over time
fig = go.Figure()

# Red wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_Red_Demand_1']*1000,
    name="Red Wine Demand",
    marker_color="red"
))

# White wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_White_Demand_1']*1000,
    name="White Wine Demand",
    marker_color="lightgreen"
))

# Red wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_Red_Demand_2']*1000,
    name="Red Wine Demand",
    marker_color="darkred"
))

# White wine demand trace
fig.add_trace(go.Bar(
    x=df['Year_Month'],
    y=df['Total_White_Demand_2']*1000,
    name="White Wine Demand",
    marker_color="green"
))

fig.update_layout(
    title="Linear Demand: Monthly Wine Demand Satisfied by Type",
    xaxis_title="Month-Year",
    yaxis_title="Wine Demand Satisfied (in Thousands)",
    xaxis={'type': 'category'},  # Ensures that months are treated as distinct categories
    barmode='stack'  # Groups the bars for red and white wine side by side for each month
)

# This setup would allow for filtering by wine type by clicking on the legend items.
fig.show()


In [306]:
# Assuming df is your DataFrame containing the data
df = df2_sim2.copy()

# Aggregating the total demand for Red and White wine across all months for each country
red_columns_1 = [col for col in df.columns if 'Red_1' in col]
white_columns_1 = [col for col in df.columns if 'White_1' in col]
red_columns_2 = [col for col in df.columns if 'Red_2' in col]
white_columns_2 = [col for col in df.columns if 'White_2' in col]

# Convert 'Month' to datetime for proper sorting and display
df['Month'] = pd.to_datetime(df['Month'])

# Aggregate monthly total demand for Red and White wine across all countries
df['Total_Red_Demand_1'] = df[red_columns_1].sum(axis=1)
df['Total_White_Demand_1'] = df[white_columns_1].sum(axis=1)
df['Total_Red_Demand_2'] = df[red_columns_2].sum(axis=1)
df['Total_White_Demand_2'] = df[white_columns_2].sum(axis=1)

# Calculate cumulative sum for each type of wine
df['Cumulative_Red_Demand_1'] = df['Total_Red_Demand_1'].cumsum() * 125 # 'Red_Wine_1': 125,
df['Cumulative_White_Demand_1'] = df['Total_White_Demand_1'].cumsum() * 90 # 'White_Wine_1': 90,
df['Cumulative_Red_Demand_2'] = df['Total_Red_Demand_2'].cumsum() * 260 # 'Red_Wine_2': 260,
df['Cumulative_White_Demand_2'] = df['Total_White_Demand_2'].cumsum() * 200 # 'White_Wine_2': 200,

# Extracting year-month for the x-axis
df['Year_Month'] = df['Month'].dt.strftime('%Y-%m')

# Create stacked line plot
fig = go.Figure()

# Stacked red wine demand 1
fig.add_trace(go.Scatter(
    x=df['Year_Month'],
    y=df['Cumulative_Red_Demand_1'],
    mode='lines',
    name='Cumulative Red Wine Demand 1',
    stackgroup='one',
    line=dict(color='red')
))

# Stacked white wine demand 1
fig.add_trace(go.Scatter(
    x=df['Year_Month'],
    y=df['Cumulative_White_Demand_1'],
    mode='lines',
    name='Cumulative White Wine Demand 1',
    stackgroup='one',
    line=dict(color='lightgreen')
))

# Stacked red wine demand 2
fig.add_trace(go.Scatter(
    x=df['Year_Month'],
    y=df['Cumulative_Red_Demand_2'],
    mode='lines',
    name='Cumulative Red Wine Demand 2',
    stackgroup='one',
    line=dict(color='darkred')
))

# Stacked white wine demand 2
fig.add_trace(go.Scatter(
    x=df['Year_Month'],
    y=df['Cumulative_White_Demand_2'],
    mode='lines',
    name='Cumulative White Wine Demand 2',
    stackgroup='one',
    line=dict(color='green')
))

fig.update_layout(
    title="Linear Demand: Expected Profit in Simulation 2",
    xaxis_title="Month",
    yaxis_title="Expected Profit in Euros (in Millions)",
    hovermode="x unified",
    xaxis={'type': 'category'}  # Ensures that months are treated as distinct categories
)

fig.show()
